# 匯出數據

In [6]:
import pandas as pd
from datetime import datetime
import os
import pymysql
from dotenv import load_dotenv

load_dotenv()
DB_HOST = os.getenv("DB_HOST")
DB_PORT = int(os.getenv("DB_PORT", 3306))
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")


# 自訂函數
def query_products(
    start_date=None, 
    end_date=None, 
    limit=None, 
    offset=None,
    # 如：'output.csv' 或 'output.xlsx'
    export_path=None
):
    connection = pymysql.connect(
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset='utf8mb4',
        # 回傳 dict 格式
        cursorclass=pymysql.cursors.DictCursor
    )

    with connection:
        with connection.cursor() as cursor:
            sql = "SELECT id, search_keyword, title, price, unit_price, full_text, timestamp FROM coupang_products WHERE 1=1"
            params = []

            # 日期篩選
            if start_date and end_date:
                sql += " AND timestamp BETWEEN %s AND %s"
                params.extend([start_date, end_date])

            # 排序
            sql += " ORDER BY id DESC"

            # 分頁功能
            if limit:
                sql += " LIMIT %s"
                params.append(limit)
                if offset:
                    sql += " OFFSET %s"
                    params.append(offset)

            cursor.execute(sql, params)
            rows = cursor.fetchall()

            # 顯示資料
            print("查詢結果：")
            for row in rows:
                print(f"ID: {row['id']}")
                print(f"關鍵字: {row['search_keyword']}")
                print(f"標題: {row['title']}")
                print(f"價格: {row['price']}")
                print(f"每單位: {row['unit_price']}")
                print(f"完整內容: {row['full_text']}")
                print(f"時間戳記: {row['timestamp']}")
                print("-" * 80)

            # 匯出功能
            if export_path:
                df = pd.DataFrame(rows)
                if export_path.endswith(".csv"):
                    df.to_csv(
                        export_path,
                        index=False,
                        encoding="utf-8-sig"
                    )
                    print(f"資料已匯出為 CSV：{export_path}")
                elif export_path.endswith(".xlsx"):
                    df.to_excel(export_path, index=False)
                    print(f"資料已匯出為 Excel：{export_path}")
                else:
                    print("不支援的檔案格式，只支援 .csv 與 .xlsx")

if __name__ == "__main__":
    # 篩選日期
    start_date = "2024-12-01"
    end_date = "2025-12-31"

    # 分頁參數
    # 資料筆數
    limit = 20
    # 資料起點序號
    offset = 0

    # 匯出路徑，以下指定為 `coupang_output.csv`
    # 設定為 `None`` 表示不匯出
    export_path = "coupang_output.xlsx"

    # 執行查詢
    query_products(
        start_date=start_date,
        end_date=end_date,
        limit=limit,
        offset=offset,
        export_path=export_path
    )


查詢結果：
ID: 484
關鍵字: 味丹 氣泡水
標題: 多喝水 氣泡水, 1.25L, 60瓶
價格: $1,991
每單位: ($2.66/100ml)
完整內容: 多喝水 氣泡水, 1.25L, 60瓶 特價 8折 $2,520 $1,991 ($2.66/100ml) 7折 優惠券 缺貨 免運 ( 919 )
時間戳記: 2025-05-16 23:06:12
--------------------------------------------------------------------------------
ID: 483
關鍵字: 味丹 氣泡水
標題: Cloop 零卡氣泡飲 柚子口味, 6罐, 500ml
價格: $150
每單位: ($5.00/100ml)
完整內容: Cloop 零卡氣泡飲 柚子口味, 6罐, 500ml 特價 43折 $352 $150 ($5.00/100ml) 5/26 預計送達 免運 滿 $690 ( 196 )
時間戳記: 2025-05-16 23:06:12
--------------------------------------------------------------------------------
ID: 482
關鍵字: 味丹 氣泡水
標題: 味丹 究.選 SUAN 草莓紅茶氣泡飲, 540ml, 72瓶
價格: $1,097
每單位: ($2.82/100ml)
完整內容: 味丹 究.選 SUAN 草莓紅茶氣泡飲, 540ml, 72瓶 特價 74折 $1,497 $1,097 ($2.82/100ml) 7折 優惠券 5/19 星期一 預計送達 免運 ( 22 )
時間戳記: 2025-05-16 23:06:12
--------------------------------------------------------------------------------
ID: 481
關鍵字: 味丹 氣泡水
標題: 味丹 究‧選 SUÁN蜜桃紅茶氣泡飲, 540ml, 72瓶
價格: $1,352
每單位: ($3.48/100ml)
完整內容: 味丹 究‧選 SUÁN蜜桃紅茶氣泡飲, 540ml, 72瓶 特價 65折 $2,088 $1,352 ($3.48/100ml